In [25]:
from keras.models import Model, Sequential
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.layers import Dense, Flatten, Dropout, Input, BatchNormalization
from keras.layers import Convolution2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import ELU
from keras.optimizers import Adam
from keras import backend as K
import numpy as np
import os
import random
import pandas as pd
from skimage.io import imread
from keras.callbacks import EarlyStopping, ModelCheckpoint
from scipy import ndimage

from sklearn.model_selection import train_test_split


In [9]:
image_size = (480,640) #or (299, 299)
if K.image_dim_ordering() == 'th':
    input_shape = (3,) + image_size
else:
    input_shape = image_size + (3, )
    bottleneck_model = InceptionV3(weights='imagenet',
                                   include_top=False,
                                   input_tensor=Input(input_shape))


In [10]:
for layer in bottleneck_model.layers:
    layer.trainable = False

In [11]:
x = bottleneck_model.input
y = bottleneck_model.output
# There are different ways to handle the bottleneck output
y = GlobalAveragePooling2D()(x)
#y = AveragePooling2D((8, 8), strides=(8, 8))(x)
#y = Flatten()(y)
#y = BatchNormalization()(y)
y = Dense(1024, activation='relu')(y)
y = Dropout(.5)(y)
y = Dense(1)(y)

model = Model(input=x, output=y)
model.compile(optimizer=Adam(lr=1e-4), loss = 'mse')

/Users/sullivan42/anaconda3/envs/cs231n/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  if sys.path[0] == '':


In [12]:
print(model)

In [13]:
dataset_path = '/Users/sullivan42/workspace/cs231n_proj/cs231n_final_proj/data/output2'
camera = 'center'
steering_log = os.path.join(dataset_path, 'steering.csv')
image_log = os.path.join(dataset_path, 'camera.csv')
cam_path = '/Users/sullivan42/workspace/cs231n_proj/cs231n_final_proj/data/output2/center'


In [26]:
df=pd.read_csv(os.path.join(dataset_path,'interpolated.csv'), sep=',')
h = df.head(1302)
one = df.sample(1)
train, test = train_test_split(df, test_size=0.2)

In [15]:
dataset_path = '/Users/sullivan42/workspace/cs231n_proj/cs231n_final_proj/data/output2'

imgs = []
angles = []
for index, row in h.iterrows():
    #print(row['frame_id'])
    if((row['frame_id'] == 'center_camera')):
        #print(row['filename'])
        fname = row['filename'] 
        img = imread(os.path.join(dataset_path, "{}".format(fname))) # (480,640,3) np array
        imgs.append(img)
        angles.append(row['angle'])

In [16]:
img_block = np.stack(imgs, axis=0)
angles_block = np.stack(angles, axis=0)

In [17]:
model.fit(x=img_block, y=angles_block)


Epoch 1/1
434/434 [==============================] - 3s 6ms/step - loss: 75.0420


In [27]:
def generator(df, mode, batch_size):
    dataset_path = '/Users/sullivan42/workspace/cs231n_proj/cs231n_final_proj/data/output2'
    imgs = []
    angles = []
    while True:
        while len(angles) < batch_size:
            # choose random index in features
            row = df.sample(n=1, replace=False, weights=None)
            id = row['frame_id'].astype(str)
            t = id == 'center_camera'
            #print(t.item())
            is_center = t.item()
            is_left =  (id == 'left_camera').item()
            is_right =  (id == 'left_camera').item()

            if(True):
                #print(row['filename'])
                fname = row['filename'].item()
                img = imread(os.path.join(dataset_path, "{}".format(fname))) # (480,640,3) np array
                angle = row['angle']
                if mode == 'xval' or mode == 'test':
                    if is_center:
                        imgs.append(img)
                        angles.append(angle) 
                else:
                    do_flip = random.randint(0, 1)
                    do_brighten = random.randint(-1, 1)
                    do_rotate = random.randint(0, 1)
                    if is_left:
                        angle += .25
                    if is_right:
                        angle -= .25
                        dy = 25 #in
                        speed = row['speed']
                        tr = 1#s
                        #angle += np.arctan(dy/tr)
                        #of car wandering off to the side, 
                        #and recovering. We will add a small angle .25 
                        #to the left camera and subtract a small angle of 
                        #0.25 from the right camera. The main idea being the
                        #left camera has to move right to get to center,
                        #and right camera has to move left.
                    if do_flip > 0:
                        angle *= -1
                        img = np.fliplr(img)
                    if do_rotate:
                        #rotate without affecting steering
                        rotate = random.uniform(-1, 1)
                        img = ndimage.rotate(img, rotate, reshape=False)
                    if do_brighten:
                        #blurr
                        img = ndimage.gaussian_filter(img, sigma=3)
                    imgs.append(img)
                    angles.append(angle) 
        img_block = np.stack(imgs, axis=0)
        angles_block = np.stack(angles, axis=0)
        yield img_block, angles_block

In [32]:
#model.fit_generator(generator(df, 50), samples_per_epoch=50, nb_epoch=10)

#xval = yield img_block,angles_block

callbacks = [EarlyStopping(monitor='val_loss', patience=3, verbose=0),
                    ModelCheckpoint(filepath=os.path.join('weights_HMB.hdf5'),
                    monitor='val_loss', verbose=0, save_best_only=True)]


"""model.fit_generator(train_generator, samples_per_epoch=6400, nb_epoch=nb_epoch,verbose=1,
                    callbacks=callbacks, validation_data=val_generator,nb_val_samples=1600)"""

model.fit_generator(generator(train,'train',50), samples_per_epoch=140*4, nb_epoch=12,verbose=1,
                    callbacks=callbacks, validation_data=generator(train,'xval',50),nb_val_samples=60*4)

/Users/sullivan42/anaconda3/envs/cs231n/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
/Users/sullivan42/anaconda3/envs/cs231n/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., verbose=1, callbacks=[<keras.ca..., validation_data=<generator..., steps_per_epoch=560, epochs=12, validation_steps=240)`
  


Epoch 1/12
560/560 [==============================] - 130s 232ms/step - loss: 0.0020 - val_loss: 0.0049
Epoch 2/12
560/560 [==============================] - 117s 209ms/step - loss: 0.0018 - val_loss: 0.0046
Epoch 3/12
560/560 [==============================] - 117s 209ms/step - loss: 0.0017 - val_loss: 0.0046
Epoch 4/12
560/560 [==============================] - 117s 209ms/step - loss: 0.0017 - val_loss: 0.0046
Epoch 5/12
560/560 [==============================] - 118s 210ms/step - loss: 0.0016 - val_loss: 0.0047
Epoch 6/12
560/560 [==============================] - 119s 212ms/step - loss: 0.0016 - val_loss: 0.0047
Epoch 7/12
560/560 [==============================] - 118s 211ms/step - loss: 0.0016 - val_loss: 0.0046


In [30]:
scores = model.evaluate_generator(generator(test,'test',50),514) #1514 testing images
print("Accuracy = ", scores[1])

IndexError: invalid index to scalar variable.

In [31]:
print(scores)

0.005845813546329737
